<a href="https://colab.research.google.com/github/Elmar1101/Data_science_projects/blob/main/Loandata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("taweilo/loan-approval-classification-data")

print("Path to dataset files:", path)

100%|██████████| 751k/751k [00:00<00:00, 33.4MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/taweilo/loan-approval-classification-data/versions/1


In [ ]:
!kaggle datasets download -d taweilo/loan-approval-classification-data

Dataset URL: https://www.kaggle.com/datasets/taweilo/loan-approval-classification-data
License(s): apache-2.0
  0% 0.00/751k [00:00<?, ?B/s]
100% 751k/751k [00:00<00:00, 28.8MB/s]


In [ ]:
!unzip /content/loan-approval-classification-data.zip

Archive:  /content/loan-approval-classification-data.zip
  inflating: loan_data.csv           


In [ ]:
import pandas as pd

df=pd.read_csv('/content/loan_data.csv')

In [ ]:
df.head(5)

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


In [ ]:
df.drop(['person_age','person_gender','person_education'],axis=1,inplace=True)

In [ ]:
df.drop(['loan_amnt','loan_intent','loan_int_rate'],axis=1,inplace=True)

In [ ]:
df.head()

,person_income,person_emp_exp,person_home_ownership,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,71948.0,0,RENT,0.49,3.0,561,No,1
1,12282.0,0,OWN,0.08,2.0,504,Yes,0
2,12438.0,3,MORTGAGE,0.44,3.0,635,No,1
3,79753.0,0,RENT,0.44,2.0,675,No,1
4,66135.0,1,RENT,0.53,4.0,586,No,1


In [ ]:
df.loan_status.isna().sum()

0

In [ ]:
X=df.drop('loan_status',axis=1)
y=df['loan_status'].copy()

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:
import numpy as np
num_features=X_train.select_dtypes(include=[np.number]).columns
cat_features=X_train.select_dtypes(exclude=[np.number]).columns

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

cat_pipeline=Pipeline([
    ('impute',SimpleImputer(strategy='constant',fill_value='Missing')),
    ('one_hot',OneHotEncoder(handle_unknown='ignore',sparse_output=False))
])

num_pipeline=Pipeline([
    ('impute',SimpleImputer(strategy='median')),
    ('scale',StandardScaler())
])

In [ ]:
from sklearn.compose import ColumnTransformer
transformer = ColumnTransformer([
    ('num',num_pipeline,num_features),
    ('cat',cat_pipeline,cat_features)
],remainder='passthrough')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
estimator = RandomForestClassifier()
full_pipeline=Pipeline([
    ('preprocessing',transformer),
    ('estimator',estimator)
])

In [ ]:
full_pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  Index(['person_income', 'person_emp_exp', 'loan_percent_income',
       'cb_person_cred_hist_length', 'credit_score'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(fill_value='Missing',
                                                                                 strategy='constant')),
                                                                  ('one_hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  Index(['person_home_ownership', 'previous_loan_defaults_on_file'], dtype='object'))])),
                ('estimator', RandomForestClassifier())])

In [ ]:
full_pipeline.score(X_test,y_test)

0.8832222222222222

In [ ]:
full_pipeline.score(X_train,y_train)

1.0

In [ ]:
full_pipeline.predict(X_test)

array([0.2 , 0.  , 0.57, ..., 0.17, 0.66, 0.69])

In [ ]:
# from sklearn.linear_model import LinearRegression
# reg=LinearRegression().fit(X_train,y_train )
# reg.score(X_test,y_test)

ValueError: could not convert string to float: 'MORTGAGE'